In [1]:
import matplotlib as mlp
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras 
import warnings

warnings.filterwarnings('ignore')
print(tf.__version__)
for model in sklearn, pd, keras, np ,mlp:
    print(model.__name__, model.__version__)

2.1.0
sklearn 0.20.2
pandas 0.24.2
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
numpy 1.17.4
matplotlib 2.1.2


In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
housing = fetch_california_housing()
x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state=1)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state=2)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [3]:
# version 0.2 数据归一化
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
x_train = std.fit_transform(x_train)
x_valid = std.transform(x_valid)
x_test = std.transform(x_test)
print(x_train.shape[1:])

(8,)


In [4]:
"""
# 函数式 API
input = keras.layers.Input(shape=x_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation='relu')(input)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)

# 假设 wide部分 和deep 部分的输入是一样的，因此需要将数据进行合并
concat = keras.layers.concatenate([input, hidden2])
output = keras.layers.Dense(1)(concat)

# 将写好的网络结构固化下来
model = keras.models.Model(inputs=[input],
                           outputs=[output])
"""


# # 使用子类API的方式
# class WideDeepModel(keras.models.Model):
#     def __init__(self):
#         super(WideDeepModel,self).__init__()
#         """定义模型的层次"""
#         self.hidden1 = keras.layers.Dense(30, activation='relu')
#         self.hidden2 = keras.layers.Dense(30, activation='relu')
#         self.output_layer = keras.layers.Dense(1)
            
#     def call(self, input):
#         """完成模型的正向计算"""
#         hidden1 = self.hidden1(input)
#         hidden2 = self.hidden2(hidden1)
#         concat = keras.layers.concatenate([input, hidden2])
#         output = self.output_layer(concat)
#         return output
    
# model = WideDeepModel()
# model.build(input_shape=(None, 8))


# 多输入，函数式 API
input_wide = keras.layers.Input(shape=[5])
input_deep = keras.layers.Input(shape=[6])
hidden1 = keras.layers.Dense(30, activation='relu')(input_deep)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.concatenate([input_wide,hidden2])
output = keras.layers.Dense(1)(concat)

model = keras.models.Model(inputs=[input_wide, input_deep],
                           outputs = [output])




# sparse_categorical_crossentropy:  如果y是一个数，会把它转化成一个向量，如 y->index., y->one hot->[]
model.compile(loss='mean_squared_error',
              optimizer='adam')


SyntaxError: invalid syntax (<ipython-input-4-4ad53ca19d44>, line 21)

In [ ]:
# version 0.3 回调函数
# Tensorboard, Earlystopping, Modelcheckpoint
logdir = './wide_deep_callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,
                                 "housing_model.h5")

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file, save_best_only=True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)
]

In [ ]:
model.summary()

In [ ]:
"""
# 单输入训练
history = model.fit(x_train, y_train, epochs=10,validation_data=(x_valid, y_valid),callbacks=callbacks)
"""

# 多输入训练
x_train_wide = x_train[:, :5]
x_train_deep = x_train[:, 2:]
x_valid_wide = x_valid[:, :5]
x_valid_deep = x_valid[:, 2:]
x_test_wide = x_test[:, :5]
x_test_deep = x_test[:, 2:]
history = model.fit([x_train_wide, x_train_deep],
                    y_train,
                    epochs=10,
                    validation_data=([x_valid_wide, x_valid_deep],
                                     y_valid),
                    callbacks=callbacks)


In [ ]:
def plot_learning_curve(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()
    
plot_learning_curve(history)

In [ ]:
model.evaluate([x_test_wide, x_test_deep],
               y_test)